In [1]:
!rm enron1.zip*
!wget https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/enron1.zip
!unzip -q enron1.zip
!ls -la
!ls -la enron1
!pip install optuna

rm: cannot remove 'enron1.zip*': No such file or directory
--2022-03-05 05:35:56--  https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/enron1.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oreilly-japan/ml-security-jp/master/ch02/enron1.zip [following]
--2022-03-05 05:35:57--  https://raw.githubusercontent.com/oreilly-japan/ml-security-jp/master/ch02/enron1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3597958 (3.4M) [application/zip]
Saving to: ‘enron1.zip’

enron1.zip          100%[===================>]   3.43M  --.-KB/s    in 0.01s   

2022-03-05 05:35:57 (256 MB/s) - 

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold, cross_validate
import os
import codecs

def init_lists(folder):
    key_list = []
    file_list = os.listdir(folder)
    for filename in file_list:
        f = codecs.open(folder + filename, 'r', encoding='utf8', errors='ignore')
        key_list.append(f.read())
    f.close()
    return key_list

all_mails = list()
spam = init_lists('./enron1/spam/')
ham = init_lists('./enron1/ham/')

all_mails = [(mail, '1') for mail in spam]
all_mails + [(mail, '0') for mail in ham]

import pandas as pd
df = pd.DataFrame(all_mails, columns=['text', 'label'])

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english")
X = tfidf.fit_transform(df['text'])
column_names = tfidf.get_feature_names()

X = pd.DataFrame(X.toarray())
X = X.astype('float')

X.columns = column_names
y = df['label'].astype('float')

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as olgb
import optuna
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=101)

train = olgb.Dataset(X_train, y_train)

params = {
      "objective": "binary",
      "verbosity": -1,
      "bosting_type": "gbdt"
}

tuner = olgb.LightGBMTunerCV(params, train, num_boost_round=100)

tuner.run()

print("Best score:", 1 - tuner.best_score)
best_params = tuner.best_params

print("Best Params:")
for key, value in best_params.items():
    print("     {}: {}".format(key, value))

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'binary',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'lambdal1': best_params['lambda_l1'],
    'lambdal2': best_params['lambda_l2'],
    'num_leaves': best_params['num_leaves'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'bagging_freq': best_params['bagging_freq'],
    'min_child_samples': best_params['min_child_samples']
}

gbm = lgb.train(
    params,
    train_data,
    num_boost_round=100,
    verbose_eval=0
)

preds = gbm.predict(X_test)
pred_labels = np.rint(preds)
print("Accuracy: {:.5f} %".format(100 * accuracy_score(y_test, pred_labels)))
print(confusion_matrix(y_test, pred_labels))

import matplotlib.pyplot as plt
lgb.plot_importance(gbm, figsize=(12, 6), max_num_features=10)
plt.show()

spam_rows = (df.label == '1')
spam_data = df[spam_rows]

count = 0
for i in spam_data['text']:
  count = count + i.count('subject')

print(count)

legit_rows = (df.label == '0')
legit_data = df[legit_rows]

count = 0
for i in legit_data['text']:
  count = count + i.count('subject')

print(count)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
[I 2022-03-05 07:35:43,487] A new study created in memory with name: no-name-3655f7cc-46ab-443f-bf5b-d160d1a85125
feature_fraction, val_score: 0.000000:  14%|#4        | 1/7 [00:01<00:10,  1.68s/it]

[1]	cv_agg's binary_logloss: 0 + 0
[2]	cv_agg's binary_logloss: 0 + 0
[3]	cv_agg's binary_logloss: 0 + 0
[4]	cv_agg's binary_logloss: 0 + 0
[5]	cv_agg's binary_logloss: 0 + 0
[6]	cv_agg's binary_logloss: 0 + 0
[7]	cv_agg's binary_logloss: 0 + 0
[8]	cv_agg's binary_logloss: 0 + 0
[9]	cv_agg's binary_logloss: 0 + 0
[10]	cv_agg's binary_logloss: 0 + 0
[11]	cv_agg's binary_logloss: 0 + 0
[12]	cv_agg's binary_logloss: 0 + 0
[13]	cv_agg's binary_logloss: 0 + 0
[14]	cv_agg's binary_logloss: 0 + 0
[15]	cv_agg's binary_logloss: 0 + 0
[16]	cv_agg's binary_logloss: 0 + 0
[17]	cv_agg's binary_logloss: 0 + 0
[18]	cv_agg's binary_logloss: 0 + 0
[19]	cv_agg's binary_logloss: 0 + 0
[20]	cv_agg's binary_logloss: 0 + 0
[21]	cv_agg's binary_logloss: 0 + 0
[22]	cv_agg's binary_logloss: 0 + 0
[23]	cv_agg's binary_logloss: 0 + 0
[24]	cv_agg's binary_logloss: 0 + 0
[25]	cv_agg's binary_logloss: 0 + 0
[26]	cv_agg's binary_logloss: 0 + 0
[27]	cv_agg's binary_logloss: 0 + 0
[28]	cv_agg's binary_logloss: 0 + 0
[

feature_fraction, val_score: 0.000000:  29%|##8       | 2/7 [00:03<00:07,  1.54s/it]

[1]	cv_agg's binary_logloss: 0 + 0
[2]	cv_agg's binary_logloss: 0 + 0
[3]	cv_agg's binary_logloss: 0 + 0
[4]	cv_agg's binary_logloss: 0 + 0
[5]	cv_agg's binary_logloss: 0 + 0
[6]	cv_agg's binary_logloss: 0 + 0
[7]	cv_agg's binary_logloss: 0 + 0
[8]	cv_agg's binary_logloss: 0 + 0
[9]	cv_agg's binary_logloss: 0 + 0
[10]	cv_agg's binary_logloss: 0 + 0
[11]	cv_agg's binary_logloss: 0 + 0
[12]	cv_agg's binary_logloss: 0 + 0
[13]	cv_agg's binary_logloss: 0 + 0
[14]	cv_agg's binary_logloss: 0 + 0
[15]	cv_agg's binary_logloss: 0 + 0
[16]	cv_agg's binary_logloss: 0 + 0
[17]	cv_agg's binary_logloss: 0 + 0
[18]	cv_agg's binary_logloss: 0 + 0
[19]	cv_agg's binary_logloss: 0 + 0
[20]	cv_agg's binary_logloss: 0 + 0
[21]	cv_agg's binary_logloss: 0 + 0
[22]	cv_agg's binary_logloss: 0 + 0
[23]	cv_agg's binary_logloss: 0 + 0
[24]	cv_agg's binary_logloss: 0 + 0
[25]	cv_agg's binary_logloss: 0 + 0
[26]	cv_agg's binary_logloss: 0 + 0
[27]	cv_agg's binary_logloss: 0 + 0
[28]	cv_agg's binary_logloss: 0 + 0
[

feature_fraction, val_score: 0.000000:  43%|####2     | 3/7 [00:04<00:05,  1.49s/it]

...

[1]	cv_agg's binary_logloss: 0 + 0
[2]	cv_agg's binary_logloss: 0 + 0
[3]	cv_agg's binary_logloss: 0 + 0
[4]	cv_agg's binary_logloss: 0 + 0
[5]	cv_agg's binary_logloss: 0 + 0
[6]	cv_agg's binary_logloss: 0 + 0
[7]	cv_agg's binary_logloss: 0 + 0
[8]	cv_agg's binary_logloss: 0 + 0
[9]	cv_agg's binary_logloss: 0 + 0
[10]	cv_agg's binary_logloss: 0 + 0
[11]	cv_agg's binary_logloss: 0 + 0
[12]	cv_agg's binary_logloss: 0 + 0
[13]	cv_agg's binary_logloss: 0 + 0
[14]	cv_agg's binary_logloss: 0 + 0
[15]	cv_agg's binary_logloss: 0 + 0
[16]	cv_agg's binary_logloss: 0 + 0
[17]	cv_agg's binary_logloss: 0 + 0
[18]	cv_agg's binary_logloss: 0 + 0
[19]	cv_agg's binary_logloss: 0 + 0
[20]	cv_agg's binary_logloss: 0 + 0
[21]	cv_agg's binary_logloss: 0 + 0
[22]	cv_agg's binary_logloss: 0 + 0
[23]	cv_agg's binary_logloss: 0 + 0
[24]	cv_agg's binary_logloss: 0 + 0
[25]	cv_agg's binary_logloss: 0 + 0
[26]	cv_agg's binary_logloss: 0 + 0
[27]	cv_agg's binary_logloss: 0 + 0
[28]	cv_agg's binary_logloss: 0 + 0
[

min_data_in_leaf, val_score: 0.000000: 100%|##########| 5/5 [00:08<00:00,  1.63s/it]

[1]	cv_agg's binary_logloss: 0 + 0
[2]	cv_agg's binary_logloss: 0 + 0
[3]	cv_agg's binary_logloss: 0 + 0
[4]	cv_agg's binary_logloss: 0 + 0
[5]	cv_agg's binary_logloss: 0 + 0
[6]	cv_agg's binary_logloss: 0 + 0
[7]	cv_agg's binary_logloss: 0 + 0
[8]	cv_agg's binary_logloss: 0 + 0
[9]	cv_agg's binary_logloss: 0 + 0
[10]	cv_agg's binary_logloss: 0 + 0
[11]	cv_agg's binary_logloss: 0 + 0
[12]	cv_agg's binary_logloss: 0 + 0
[13]	cv_agg's binary_logloss: 0 + 0
[14]	cv_agg's binary_logloss: 0 + 0
[15]	cv_agg's binary_logloss: 0 + 0
[16]	cv_agg's binary_logloss: 0 + 0
[17]	cv_agg's binary_logloss: 0 + 0
[18]	cv_agg's binary_logloss: 0 + 0
[19]	cv_agg's binary_logloss: 0 + 0
[20]	cv_agg's binary_logloss: 0 + 0
[21]	cv_agg's binary_logloss: 0 + 0
[22]	cv_agg's binary_logloss: 0 + 0
[23]	cv_agg's binary_logloss: 0 + 0
[24]	cv_agg's binary_logloss: 0 + 0
[25]	cv_agg's binary_logloss: 0 + 0
[26]	cv_agg's binary_logloss: 0 + 0
[27]	cv_agg's binary_logloss: 0 + 0
[28]	cv_agg's binary_logloss: 0 + 0
[

Accuracy: 100.00000 %
[[300]]


ValueError: ignored